## Part 1: Tools to process HTML

In Part 0, you downloaded real web pages and manipulated them using "conventional" string processing tools, like [`str`]() functions or [regular expressions]().

However, web pages are stored in HTML ([hypertext markup language]()), which is a structured format. As such, it makes sense to use specialized tools to understand and process its structure. That's the subject of this notebook.

## Parsing HTML: The Beautiful Soup module

One such package to help process HTML is [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/). The following is a quick tutorial on how to use it.

Any HTML document may be modeled as an object in computer science known as a [tree](https://en.wikipedia.org/wiki/Tree_(data_structure)):

![HTML as a tree](./html-slide.png)

There are different ways to define trees, but for our purposes, the following will be sufficient.

Consider a tree is a collection of _nodes_, which are the labeled boxes in the figure, and _edges_, which are the line segments connecting nodes, with the following special structure.

* The node at the top is called the _root_. Here, the root is labeled `html` and abstractly represents the entire HTML document.
* Regard each edge as always "pointing" from the node at its top end to the node at its bottom end. For any edge, the node at its top end is the _parent_ and the node at the bottom end is a _child_. Like real families, a parent can be a child. For example, the node labeled `head` is the child of `html` and the parent of `meta`, `title`, and `style`.
* The _descendant_ of a node $x$ is any node $y$ for which there is a path from $x$ going down to $y$. For example, the node labeled `6x span` is a descendant of the node `body`. All nodes are descendants of the root.
* Any node with _no_ descendants is a _leaf_.
* Any node that is neither a root nor a leaf is an _internal node_.
* There are no _cycles_. A cycle would be a loop. For instance, if you were to add an edge between the two lower rightmost nodes labeled, `strong` and `strong`, that would create a loop and the object would no longer be a tree.

> For whatever reason, [computer scientists usually view trees upside down](https://www.quora.com/Why-are-trees-in-computer-science-generally-drawn-upside-down-from-how-trees-are-in-real-life), with the "root" at the top and the "leaves" at the bottom.

The Beautiful Soup package gives you a data structure for traversing this tree. For instance, consider an HTML file with the contents below, shown both as code and pictorially.

In [ ]:
some_page = """
<html>
  <body>
    <p>First paragraph.</p>
    <p>Second paragraph, which links to the <a href="http://www.gatech.edu">Georgia Tech website</a>.</p>
    <p>Third paragraph.</p>
  </body>
</html>
"""
print(some_page)

![Two visual representations of `some_page`](./html-viz.png)

**Exercise 0.** Besides HTML files, what else have we seen in this class that could be represented by a tree? Briefly and roughly explain what and how.

**Answer.** One thing that has a natural tree representation is a Python program! For example, can you draw the following program as a tree?

```python
import re

def scan_lines(text, pattern):
    matches = []
    for line in text.split('\n'):
        if re.search(pattern, text) is not None:
            matches.append(True)
        else:
            matches.append(False)
    return matches
```

## Using Beautiful Soup

Here is how you might use Beautiful Soup to inspect the structure of `some_page`.

Let's start by taking the contents of the page above (`some_page`) and asking Beautiful Soup to process it. Let's store the result in object named `soup`, and then explore its contents:

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(some_page, "lxml")

print('1. soup ==', soup) # Print the HTML contents
print('\n2. soup.html ==', soup.html) # Root of the tree
print('\n3. soup.html.body ==', soup.html.body) # A child tag
print('\n4. soup.html.body.p ==', soup.html.body.p) # Another child tag
print('\n5. soup.html.body.contents ==', type(soup.html.body.contents), '::', soup.html.body.contents)

Observe that the `.` notation allows us to reference HTML tags---that is, the stuff enclosed in angle brackets in the original HTML, e.g., `<html> ... </html>`, `<body> ... </body>`---as they are nested. But in the case of the `<body> ... </body>` tag, there are multiple subtags. Evidently, `soup.html.body.contents` contains these, as a list, which we know how to manipulate.

In [ ]:
# Enumerate all tags within the <body> ... </body> tag:
for i, elem in enumerate (soup.html.body.contents):
    print ("[{:4d}]".format (i), type (elem), '\n\t==>', "'{}'".format (elem))

# Reference one of these, element 3:
elem3 = soup.html.body.contents[3]
print(elem3.contents)

**Exercise 1.** Write a statement that navigates to the tag representing the GT website link. Store this resulting tag object in a variable called `link`.

In [ ]:
###
### YOUR CODE HERE
###

print(link)

In [ ]:
# Checks your link. Can you understand what it is doing?
import bs4
assert type(link) is bs4.element.Tag
assert link.name == 'a'
assert link['href'] == 'http://www.gatech.edu'
assert link.contents == ['Georgia Tech website']

print("\n(Passed!)")

### Other navigation tools

This lab includes a static copy of the Yelp! results for a search of "universities" in ATL. Let's start by downloading this file.

In [ ]:
# Run me: Code to download sample HTML file

import requests
import os
import hashlib

yelp_htm = 'yelp_unies_atl--20200126--1347--prettified.html'
yelp_htm_checksum = 'd94da820bd8def4c1b240d272a3b673c'

if os.path.exists('.voc'):
    data_url = 'https://cse6040.gatech.edu/datasets/yelp-example-uni/{}'.format(yelp_htm)
else:
    data_url = 'https://github.com/cse6040/labs-fa17/raw/master/datasets/{}'.format(yelp_htm)

if not os.path.exists(yelp_htm):
    print("Downloading: {} ...".format(data_url))
    r = requests.get(data_url)
    with open(yelp_htm, 'w', encoding=r.encoding) as f:
        f.write(r.text)

with open(yelp_htm, 'r') as f:
    yelp_html = f.read().encode(encoding='utf-8')
    checksum = hashlib.md5(yelp_html).hexdigest()
    assert checksum == yelp_htm_checksum, "Downloaded file has incorrect checksum!"
    
print("'{}' is ready!".format(yelp_htm))

If the above ran without errorr, you can view this file by clicking this link: [./yelp_unies_atl--20200126--1347--prettified.html](./yelp_unies_atl--20200126--1347--prettified.html).

Next, inspect and run this code, which prints the top (number one) result.

In [ ]:
uni_html_text = open (yelp_htm, 'r').read()
uni_soup = BeautifulSoup(uni_html_text, "lxml")

print("The number 1 ATL university according to Yelp!:")

uni_1_span = uni_soup.html.body \
                     .contents[11].contents[7].contents[3].contents[1].contents[1].contents[1] \
                     .contents[1].contents[3].contents[1].contents[9].contents[1].contents[1].contents[1] \
                     .contents[3].contents[1].contents[1].contents[1].contents[1].contents[1].contents[1] \
                     .contents[1].contents[1].contents[1]
uni_1 = uni_1_span.contents[3]
    
print(uni_1.text.strip())

We hope it is self-evident that the above method to navigate to a particular tag or element is not terribly productive or robust, particularly if there are small modifications to the HTML.

Here is an alternative. Inspect the raw HTML and observe that every search result appears in a "span tag" of the form,
```html
    <span class="lemon--span__373c0__3997G text__373c0__2Kxyz text-color--black-regular__373c0__2vGEn text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz text-size--inherit__373c0__2fB3p">
        1
        <!-- -->
        .
        <a class="lemon--a__373c0__IEZFH link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE" href="/biz/georgia-tech-atlanta-4?osq=universities" name="Georgia Tech" rel="" target="">
            Georgia Tech
        </a>
    </span>
```
This tag has a specific class attribute (`lemon--span__373c0__3997G ...`), and for non-sponsored results, a rank (`1 <!-- --> .` indicating the first, or top-ranked, result). Beautiful Soup gives us a way to search for specific tags.

In [ ]:
target_class_value = 'lemon--span__373c0__3997G text__373c0__2Kxyz text-color--black-regular__373c0__2vGEn text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz text-size--inherit__373c0__2fB3p'
all_search_results = uni_soup.find_all(attrs={'class', target_class_value})
print("Found", len(all_search_results), "results.")
print("\n*** First three results: ***")
for k, result in enumerate(all_search_results[:3]):
    print(f"\n=== RESULT {k} ===\n")
    print(result)

**Exercise 2.** Based on the above, write a function that, given a Yelp! search results page such as `uni_soup` above, returns the name of result whose rank is `k`, an **integer**. For example, when `k == 1`, it should return, `"Georgia Tech"`; when `k == 2`, it should return, `"Spelman College"`; when `k == 3`, it should return `"Emory Continuing Education"`; and so on.

In [ ]:
def get_yelp_result(soup, k):
    """Given a Yelp! search result as a Beautiful Soup page,
    returns the name of business whose rank is `k`.
    """
    ###
    ### YOUR CODE HERE
    ###
    
# Demo:
print(get_yelp_result(uni_soup, 1))

In [ ]:
top_5 = ["Georgia Tech", "Spelman College", "Emory Continuing Education", "Life University",
         "Oglethorpe University"]
for k in range(1, len(top_5)+1):
    your_result = get_yelp_result(uni_soup, k)
    true_result = top_5[k-1]
    assert your_result == true_result, \
           f"Your rank-{k} result ('{your_result}') does not match the expected value ('{true_result}')."
    
print("\n(Passed!)")

This mini-tutorial only scratches the surface of what is possible with Beautiful Soup. As always, refer to the [package's documentation](https://www.crummy.com/software/BeautifulSoup/) for all the awesome deets!

In [1]:
!tar --exclude='*/.voc' --exclude='*/.voc.work' -chvzf notebook.tar.gz ../../*

tar: Removing leading `../../' from member names
../../asn1346286_1/
../../asn1346286_1/resource/
../../asn1346286_1/resource/startercode/
../../asn1346286_1/resource/startercode/gatech_edu--20190125-1143.html
../../asn1346286_1/resource/startercode/yelp-fried_chicken-atl--20200125-1240.html
../../asn1346286_1/resource/startercode/gatech_edu--20190125-1143.png
tar: ../../asn1346286_1/resource/startercode/resource: File removed before we read it
../../asn1346286_1/resource/startercode/yelp-ramen-atl--20200125-1205.html
../../asn1346286_1/resource/startercode/yelp-ramen-atl--20200125-1205--scroll-to-results-annotated.png
../../asn1346286_1/resource/startercode/part0.ipynb
../../asn1346286_1/resource/scripts/
../../asn1346286_1/resource/scripts/run.sh
../../asn1346286_1/resource/scripts/build.sh
../../asn1346286_1/resource/asnlib/
../../asn1346286_1/resource/asnlib/publicdata/
../../asn1346286_1/resource/asnlib/public/
../../asn1346286_1/resource/asnlib/public/docs/
../../asn1346286_1/res

../../asn1346289_3/work/.ipynb_checkpoints/
../../asn1346289_3/work/.ipython/
../../asn1346289_3/work/.ipython/nbextensions/
../../asn1346289_3/work/.ipython/extensions/
../../asn1346289_3/work/.ipython/profile_default/
../../asn1346289_3/work/.ipython/profile_default/security/
../../asn1346289_3/work/.ipython/profile_default/history.sqlite
../../asn1346289_3/work/.ipython/profile_default/log/
../../asn1346289_3/work/.ipython/profile_default/db/
../../asn1346289_3/work/.ipython/profile_default/pid/
../../asn1346289_3/work/.ipython/profile_default/startup/
../../asn1346289_3/work/.ipython/profile_default/startup/README
../../asn1346289_3/work/part2.ipynb
../../asn1346289_3/work/.local/
../../asn1346289_3/work/.local/share/
../../asn1346289_3/work/.local/share/jupyter/
../../asn1346289_3/work/.local/share/jupyter/runtime/
../../asn1346289_3/work/.local/share/jupyter/runtime/notebook_cookie_secret
../../asn1346289_3/work/.local/share/jupyter/notebook_secret
../../asn1346289_3/work/.local/